In [1]:
# Carregando algumas bibliotecas importantes
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# Importando os arquivos
jogos = pd.read_csv('../data/raw/matches_FLA2024.csv', parse_dates=['kick-off'])
gols = pd.read_csv('../data/raw/goal_scorers_2024.csv')

In [3]:
# Mostrando as 5 primeiras linhas do dataframe jogos
jogos.head()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068
1,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227
2,CA2,Nova Iguaçu-RJ,Flamengo,1,1,Carioca - Round 2,2024-01-21 18:10:00,Estádio Almeidão-PB,16428
3,AM2,Orlando City,Flamengo,1,1,International Friendly Game,2024-01-27 16:00:00,Inter&Co Stadium-EUA,22678
4,CA3,Portuguesa-RJ,Flamengo,0,0,Carioca - Round 3,2024-01-27 18:10:00,Arena das Dunas-RN,25211


In [4]:
# Mostrando as 5 ultimas linhas do dataframe jogos
jogos.tail()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance
58,LIB10,Peñarol-URU,Flamengo,0,0,Copa Libertadores - Quarterfinals - Second Leg,2024-09-26 19:00:00,Campeón del Siglo-URU,38144
59,BR27,Flamengo,Athletico-PR,1,0,Campeonato Brasileiro - Round 27,2024-09-29 20:00:00,Maracanã-RJ,26812
60,CB7,Flamengo,Corinthians,1,0,Copa do Brasil - Semifinals - First Leg,2024-10-02 21:45:00,Maracanã-RJ,47052
61,BR28,Bahia,Flamengo,0,2,Campeonato Brasileiro - Round 28,2024-10-05 20:00:00,Arena Fonte Nova-BA,43433
62,BR29,Flamengo,Fluminense,0,2,Campeonato Brasileiro - Round 29,2024-10-17 20:00:00,Maracanã-RJ,55720


In [5]:
# Mostrando as 5 primeiras linhas do dataframe jogos
gols.head()

,id_match,gols,assist
0,CA1,Léo Pereira,Arrascaeta
1,CA1,Pedro,Gerson
2,CA1,Everton Cebolinha,NaN
3,CA1,Varela,Luiz Araújo
4,AM1,Pedro,NaN


In [6]:
# Mostrando as 5 ultimas linhas do dataframe jogos
gols.tail()

,id_match,gols,assist
92,BR26,Felipe Teresa,NaN
93,BR27,Gerson,Wesley
94,CB7,Alex Sandro,Bruno Henrique
95,BR28,Ayrton Lucas,NaN
96,BR28,Alcaraz,NaN


### Criando a coluna torneio

In [7]:
# Split o nome do torneio e a rodada
lista = [i.split(' -', 1)[0] for i in jogos["tournament"].tolist()]

# Transformando em uma nova coluna chamada torneio
jogos['torneio'] = np.array(lista)

### Criando a coluna local (casa/fora)

In [8]:
def obter_locale(row):
    if row['torneio'] == 'Supercopa do Brasil' or row['torneio'] == 'Club World Championship':
        return 'Neutro'
    elif row['home_team'] == 'Flamengo':
        return 'Casa'
    else:
        return 'Fora'

    
# Aplicar a função para criar a nova coluna
jogos['locale'] = jogos.apply(obter_locale, axis=1)

### Criando a coluna resultado

In [9]:
# Função para obter resultado
def obter_resultado(row):
    if row['home_team'] == 'Flamengo':
        gols_marcados = row['home_score']
        gols_sofridos = row['away_score']
        
        if gols_marcados < gols_sofridos:
            return 'Derrota'
        elif gols_marcados > gols_sofridos:
            return 'Vitória'
        else:
            return 'Empate'
    if row['away_team'] == 'Flamengo':
        gols_marcados = row['away_score']
        gols_sofridos = row['home_score']

        if gols_marcados < gols_sofridos:
            return 'Derrota'
        elif gols_marcados > gols_sofridos:
            return 'Vitória'
        else:
            return 'Empate'

# Aplicar a função para criar a nova coluna
jogos['resultado'] = jogos.apply(obter_resultado, axis=1)

### Criando mais algumas colunas 

In [10]:
def gerar_estatisticas_gols(row):
    if row['home_team'] == 'Flamengo':
        gols_feitos_casa = row['home_score']
        gols_sofridos_casa = row['away_score']
        gols_feitos_fora = 0
        gols_sofridos_fora = 0
        marcou_gol = 'Sim' if row['home_score'] > 0 else 'Não'
        sofreu_gol = 'Sim' if row['away_score'] > 0 else 'Não'
    elif row['away_team'] == 'Flamengo':
        gols_feitos_casa = 0
        gols_sofridos_casa = 0
        gols_feitos_fora = row['away_score']
        gols_sofridos_fora = row['home_score']
        marcou_gol = 'Sim' if row['away_score'] > 0 else 'Não'
        sofreu_gol = 'Sim' if row['home_score'] > 0 else 'Não'
    else:
        gols_feitos_casa = 0
        gols_sofridos_casa = 0
        gols_feitos_fora = 0
        gols_sofridos_fora = 0
        marcou_gol = 'Não'
        sofreu_gol = 'Não'

    return pd.Series([gols_feitos_casa, gols_sofridos_casa, gols_feitos_fora, gols_sofridos_fora, marcou_gol, sofreu_gol])

jogos[['gols_casa', 'gols_sofridos_casa', 'gols_fora', 'gols_sofridos_fora', 'marcou_gol', 'sofreu_gol']] = jogos.apply(gerar_estatisticas_gols, axis=1)        

jogos['jogos_cumulativos'] = range(1, len(jogos) + 1)

#### Salvando os dataframes 

In [11]:
jogos['year'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%Y"))
jogos['month'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%b"))
jogos['weekday'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%a"))

In [12]:
jogos.to_csv('../data/processed/matches_FLA2024.csv', index=False)
gols.to_csv('../data/processed/goal_scorers_2024.csv', index=False)

In [13]:
jogos

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,torneio,...,gols_casa,gols_sofridos_casa,gols_fora,gols_sofridos_fora,marcou_gol,sofreu_gol,jogos_cumulativos,year,month,weekday
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,4,0,0,0,Sim,Não,1,2024,Jan,Wed
1,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227,International Friendly Game,...,0,0,2,0,Sim,Não,2,2024,Jan,Sun
2,CA2,Nova Iguaçu-RJ,Flamengo,1,1,Carioca - Round 2,2024-01-21 18:10:00,Estádio Almeidão-PB,16428,Carioca,...,0,0,1,1,Sim,Sim,3,2024,Jan,Sun
3,AM2,Orlando City,Flamengo,1,1,International Friendly Game,2024-01-27 16:00:00,Inter&Co Stadium-EUA,22678,International Friendly Game,...,0,0,1,1,Sim,Sim,4,2024,Jan,Sat
4,CA3,Portuguesa-RJ,Flamengo,0,0,Carioca - Round 3,2024-01-27 18:10:00,Arena das Dunas-RN,25211,Carioca,...,0,0,0,0,Não,Não,5,2024,Jan,Sat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,LIB10,Peñarol-URU,Flamengo,0,0,Copa Libertadores - Quarterfinals - Second Leg,2024-09-26 19:00:00,Campeón del Siglo-URU,38144,Copa Libertadores,...,0,0,0,0,Não,Não,59,2024,Sep,Thu
59,BR27,Flamengo,Athletico-PR,1,0,Campeonato Brasileiro - Round 27,2024-09-29 20:00:00,Maracanã-RJ,26812,Campeonato Brasileiro,...,1,0,0,0,Sim,Não,60,2024,Sep,Sun
60,CB7,Flamengo,Corinthians,1,0,Copa do Brasil - Semifinals - First Leg,2024-10-02 21:45:00,Maracanã-RJ,47052,Copa do Brasil,...,1,0,0,0,Sim,Não,61,2024,Oct,Wed
61,BR28,Bahia,Flamengo,0,2,Campeonato Brasileiro - Round 28,2024-10-05 20:00:00,Arena Fonte Nova-BA,43433,Campeonato Brasileiro,...,0,0,2,0,Sim,Não,62,2024,Oct,Sat
